In [19]:
import pandas as pd
import numpy as np

# Loading and organizing input data

data = pd.read_excel('ConabData.xlsx', sheet_name="Base de dados")

distances = pd.read_csv('Distances.csv', index_col=0)

supplyUnitsCoordinates = pd.read_excel('DistancesCONAB.xlsx', sheet_name="Coordenadas - UAs", index_col=0)

countiesCoordinates = pd.read_excel('DistancesCONAB.xlsx', sheet_name="Coordenadas - Municípios", index_col=0)

fleet_data = pd.read_excel('ConabData.xlsx', sheet_name="Veículos", header=1)[['Quantidade','Unnamed: 4','Unnamed: 5','Unnamed: 6','Localização']].iloc[1:]
fleet_data['Localização'] = fleet_data['Localização'].str.upper()
fleet_data.columns = ['Quantidade','Capacidade','Velocidade','Preço','Localização']
fleet_data['Capacidade'] = fleet_data['Capacidade'].astype(str)


In [20]:
# Available supply units:

data['UA/ORIGEM'].unique()

array(['JOÃO PESSOA/PB', 'RECIFE/PE', 'CANOAS/RS', 'TERESINA/PI',
       'MACEIÓ/AL', 'CAMPO GRANDE/MS', 'NATAL/RN', 'IRECÊ/BA',
       'COLATINA/ES', 'MARACANAÚ/CE', 'RONDONÓPOLIS/MT', 'BRASÍLIA/DF',
       'PORTO VELHO/RO', 'ROLÂNDIA/PR', 'CRATEÚS/CE', 'ANANINDEUA/PA',
       'IMPERATRIZ/MA', 'HERVAL DO OESTE/SC', 'BERNARDINO DE CAMPOS/SP',
       'FORMOSO DO ARAGUAIA/TO', 'MANAUS/AM', 'BOA VISTA/RR'],
      dtype=object)

In [21]:
# Instance type (cpp/python) - they have slightly different format because of non-ASCII characters

instanceType = 'python'

# Selecting supply units of the instance

supplyUnits = sorted(
                    ['CRATEÚS/CE',
                     'RECIFE/PE',
                     'TERESINA/PI',
                     'MACEIÓ/AL',
                     'NATAL/RN',
                     'IRECÊ/BA',
                     'MARACANAÚ/CE',
                     'IMPERATRIZ/MA',
                     'JOÃO PESSOA/PB',
                     ])

# Time windows size (in hours) - used in previous models where time windows were considered

TW = 50

# Demand multiplier, for addressing specific period

demand_factor = 1

# Service time for each tribe visited in county - in hours

ST = 0.2

In [22]:
# Counties that lead to errors

errorCounties = ['CONDE','PALMAS','FLORESTA','INAJÁ','SOBRADINHO','SÃO SEBASTIÃO','IGUATÚ']

counties = [data[(~data['MUNICIPIO DE ENTREGA'].isin(errorCounties)) & (data['UA/ORIGEM'] == supplyUnit)]['MUNICIPIO DE ENTREGA'].unique().tolist() for supplyUnit in supplyUnits]

In [23]:
# Demands for each county

demands = data[(~data['MUNICIPIO DE ENTREGA'].isin(errorCounties))].groupby(['UA/ORIGEM', 'MUNICIPIO DE ENTREGA'])['QUANTIDADE DE CESTAS (UNID)'].sum()[supplyUnits].reset_index()

demands['QUANTIDADE DE CESTAS (UNID)'] = round(demands['QUANTIDADE DE CESTAS (UNID)']*demand_factor)

In [24]:
# Fleet, velocities and prices per distance

# Fleet

fleet_list = []

velocities_list = []

prices_list = []

# Each supply unit may or may not have one or more vehicles

for supplyUnit in supplyUnits:
    
    supplyUnitfleet = fleet_data[fleet_data['Localização'] == supplyUnit]
    
    # If there are vehicles at that supply unit:
    
    if len(supplyUnitfleet) > 0:
        
        capacities = []
        
        velocities = []

        prices = []
        
        # For each vehicle type at the supply unit
        for vehicle_type in range(len(supplyUnitfleet)):
    
            capacity = int(supplyUnitfleet['Capacidade'].iloc[vehicle_type].split('-')[0])
        
            velocity = int(supplyUnitfleet['Velocidade'].iloc[vehicle_type])

            price = int(supplyUnitfleet['Preço'].iloc[vehicle_type])

            quantity = int(supplyUnitfleet['Quantidade'].iloc[vehicle_type])
        
            capacities.extend([capacity]*quantity)
            
            velocities.extend([velocity]*quantity)
            
            prices.extend([price]*quantity)
            

        fleet_list.append(capacities)
        velocities_list.append(velocities)
        prices_list.append(prices)
        
    else:
        
        fleet_list.append([])
        velocities_list.append([])
        prices_list.append([])
    
print("Fleet capacities: ",fleet_list, "\nVelocities (km/h):", velocities_list, "\nPrice/distance ($/km):", prices_list)

Fleet capacities:  [[], [], [], [550, 550, 550], [600], [700, 700, 700], [700, 700], [600, 600], []] 
Velocities (km/h): [[], [], [], [50, 50, 50], [50], [50, 50, 50], [50, 50], [50, 50], []] 
Price/distance ($/km): [[], [], [], [4, 4, 4], [4], [4, 4, 4], [4, 4], [4, 4], []]


In [25]:
# Service times

dict_ST = dict()

for county_list in counties:
    
    for county in county_list:
        
        communities = data[data['MUNICIPIO DE ENTREGA'] == county].groupby(["DATA"]).agg({'COMUNIDADE':'nunique'})['COMUNIDADE']
        
        average_number_of_communities = round(communities.mean(), 2)
        
        dict_ST[county] = round(average_number_of_communities*ST, 2)

dict_ST

{'CRATEÚS': 0.4,
 'MONSENHOR TABOSA': 3.7,
 'BOA VIAGEM': 0.5,
 'TAMBORIL': 0.8,
 'NOVO ORIENTE': 0.4,
 'QUITERIANÓPOLIS': 0.8,
 'PORANGA': 0.4,
 'SÃO BENEDITO': 0.2,
 'BARRA DO CORDA': 0.2,
 'FERNANDO FALCÃO': 0.47,
 'MONTES ALTOS': 0.2,
 'GRAJAÚ': 0.2,
 'ARAME': 0.2,
 'AMARANTE DO MARANHÃO': 0.2,
 'IMPERATRIZ': 0.2,
 'BOM JARDIM': 0.2,
 'SÃO JOSÉ DE RIBAMAR': 0.2,
 'RAPOSA': 0.2,
 'PRESIDENTE MÉDICI': 0.2,
 'SANTA INÊS': 0.4,
 'ZÉ DOCA': 0.2,
 'VIANA': 0.2,
 'JENIPAPO DOS VIEIRAS': 0.2,
 'IBOTIRAMA': 0.2,
 'SERRA DO RAMALHO': 0.2,
 'ANGICAL': 0.2,
 'BARREIRAS': 0.2,
 'SANTA RITA DE CÁSSIA': 0.2,
 'UTINGA': 0.2,
 'ILHÉUS': 0.2,
 'PAU BRASIL': 0.2,
 'EUNÁPOLIS': 0.2,
 'SÃO JOSÉ DA VITÓRIA': 0.2,
 'PORTO SEGURO': 0.2,
 'ITAMARAJU': 0.23,
 'RODELAS': 0.2,
 'BANZAÊ': 0.2,
 'GLÓRIA': 0.2,
 'EUCLIDES DA CUNHA': 0.2,
 'RIO TINTO': 0.31,
 'BAIA DA TRAIÇÃO': 0.35,
 'MARCAÇÃO': 0.37,
 'JOÃO PESSOA': 0.3,
 'MACEIÓ': 0.2,
 'TRAIPU': 0.2,
 'PORTO DA FOLHA': 0.2,
 'PORTO REAL DO COLÉGIO': 0.2,
 'JO

In [26]:
# Constructing instance

# Nodes

pickup_demands = demands[['UA/ORIGEM', 'QUANTIDADE DE CESTAS (UNID)']].copy()

delivery_demands = demands[['MUNICIPIO DE ENTREGA', 'QUANTIDADE DE CESTAS (UNID)']].copy()

delivery_demands['QUANTIDADE DE CESTAS (UNID)'] = delivery_demands['QUANTIDADE DE CESTAS (UNID)']*(-1)

# Service times

pickup_demands['SERVICE_TIME'] = [ST]*len(pickup_demands)

delivery_demands['SERVICE_TIME'] = [dict_ST[county] for county in delivery_demands['MUNICIPIO DE ENTREGA']]

# Renaming columns

pickup_demands.columns = ['NODE','DEMAND',"SERVICE_TIME"]

pickup_demands['NODE'] = [name[:name.find('/')] for name in pickup_demands['NODE']]

delivery_demands.columns = ['NODE','DEMAND',"SERVICE_TIME"]

# Joining both

df_instance = pd.concat([pickup_demands, delivery_demands]).reset_index(drop=True)

# Appending initial and final depots

depot_indexes = [pickup_demands['NODE'].ne(SU).idxmin() for SU in pickup_demands['NODE'].unique()]
depots = pickup_demands.iloc[depot_indexes].reset_index(drop=True)
depots['DEMAND'] = [0 for demand in depots['DEMAND']]
depots['SERVICE_TIME'] = [0 for demand in depots['SERVICE_TIME']]

df_instance = depots.append(df_instance).append(depots).reset_index(drop=True)

# Time windows (all the same)

df_instance["TW_OPEN"] = [0]*len(df_instance)

df_instance["TW_CLOSE"] = [TW]*len(df_instance)

In [27]:
# Changes (cpp type instance)

def convertCharacter(string):
    
    de_para = {'Ç':'C',
               'À':'A',
               'Á':'A',
               'É':'E',
               'Í':'I',
               'Ó':'O',
               'Ú':'U',
               'Ã':'A',
               'Â':'A',
               'Ê':'E',
               'Ô':'O'}
    
    for character in string:
        
        if character in de_para.keys():
            
            string = string.replace(character, de_para[character])
            
    return string

if instanceType == 'cpp':
    
    # Grouping demands of pickup/depot nodes

    pickups = df_instance[(df_instance['NODE'].isin([name[:name.find('/')] for name in supplyUnits])) & (df_instance['DEMAND'] > 0)].groupby('NODE').agg({'DEMAND':'sum'}).reset_index()

    deliveries = df_instance[df_instance['DEMAND'] < 0]

    depots = pickups.copy()

    depots['DEMAND'] = [0 for i in depots['DEMAND']]

    pickups['SERVICE_TIME'] = [ST for i in range(len(pickups))]
    
    # Are artificial depots required?
    
    artificial_depot = depots.iloc[0]

    df_instance = depots.append(pickups).append(deliveries).append(depots).append(artificial_depot).reset_index(drop=True)

    df_instance = df_instance.fillna(0)

    df_instance['TW_CLOSE'] = [10 for i in df_instance['TW_CLOSE']]
    
    df_instance['NODE'] = [convertCharacter(node) for node in df_instance['NODE']]

In [28]:
# Exporting instance data

instance_name = "INST_"

for supplyUnit in supplyUnits:
    
    instance_name += supplyUnit[:2].upper() + "_"
    
instance_name += str(len(pickup_demands))

if instanceType == 'cpp':
    
    instance_name += '_cpp'

f = open(f"{instance_name}.txt", "w")

# Writing number of requests

f.write(str(len(pickup_demands)) + "\n")

# Writing number of vehicles

f.write(str(sum([len(vehicle_capacities) for vehicle_capacities in fleet_list])) + "\n")

# Writing fleet data

# Capacities

for index, fleet_location in enumerate(fleet_list):
    
    space = ''
    
    if len(fleet_location) > 0:
        
        for vehicle_capacity in fleet_location:
        
            f.write(space + str(vehicle_capacity))
        
            space = " "
        
        if fleet_location != fleet_list[-1]:
        
            f.write(',')
            
    else:
        
        if index != (len(fleet_list) - 1):
        
            f.write(',')

        
f.write('\n')

# Velocities

for index, fleet_location in enumerate(velocities_list):
    
    space = ''
    
    if len(fleet_location) > 0:
        
        for vehicle_velocity in fleet_location:
        
            f.write(space + str(vehicle_velocity))
        
            space = " "
            
        if fleet_location != velocities_list[-1]:
        
            f.write(',')
            
    else:
        
        if index != len(velocities_list) - 1:
        
            f.write(',')

f.write('\n')

# Prices per distance

for index, fleet_location in enumerate(prices_list):
    
    space = ''
    
    if len(fleet_location) > 0:
        
        for vehicle_price in fleet_location:
        
            f.write(space + str(vehicle_price))
        
            space = " "
            
        if fleet_location != prices_list[-1]:
        
            f.write(',')
            
    else:
        
        if index != prices_list[-1]:
        
            f.write(',')
        
# Writing instance data

for row in df_instance.iterrows():
    
    f.write('\n{} {} {} {} {}'.format(row[1]['NODE'], row[1]['SERVICE_TIME'], row[1]['DEMAND'], row[1]['TW_OPEN'], row[1]['TW_CLOSE']))


f.close()

# df_instance.to_csv('instance_test.txt', header=None, index=list(df_instance.index), sep=' ', mode='a')

In [29]:
instance_name

'INST_CR_IM_IR_JO_MA_MA_NA_RE_TE_82'